<a href="https://colab.research.google.com/github/marclamberts/football-analysis/blob/main/Relationist_vs_Positional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install necessary libraries (if not already installed)
# !pip install networkx matplotlib openpyxl scikit-learn

import pandas as pd
import numpy as np
import networkx as nx
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define a directory in Google Drive for saving results
drive_path = '/content/drive/My Drive/'

# Create the directory if it doesn't exist
import os
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# Step 2: Upload the Dataset
from google.colab import files
uploaded = files.upload()  # Upload BM2425.xlsx file

# Step 3: Load the Dataset
file_name = list(uploaded.keys())[0]  # Get the uploaded file name
data = pd.ExcelFile(file_name)
df = data.parse('Sheet1')

# Step 4: Data Preparation
df_cleaned = df[['eventId', 'typeId', 'contestantId', 'outcome', 'x', 'y', 'timeMin', 'timeSec', 'playerName']].copy()
df_cleaned.rename(columns={
    'eventId': 'event_id',
    'typeId': 'event_type',
    'contestantId': 'team_id',
    'outcome': 'event_outcome',
    'x': 'x_pos',
    'y': 'y_pos',
    'playerName': 'player_name'
}, inplace=True)
df_cleaned.dropna(subset=['player_name', 'event_type', 'x_pos', 'y_pos'], inplace=True)

# Step 5: Positional Entropy
def compute_positional_entropy(df, player_col='player_name', x_col='x_pos', y_col='y_pos'):
    from scipy.stats import entropy
    player_positions = df.groupby(player_col).apply(
        lambda player_df: np.histogram2d(
            player_df[x_col], player_df[y_col], bins=20, range=[[0, 100], [0, 100]]
        )[0]
    )
    entropies = {player: entropy(hist.ravel()) for player, hist in player_positions.items()}
    return entropies

positional_entropies = compute_positional_entropy(df_cleaned)

# Save positional entropies to a DataFrame
entropy_df = pd.DataFrame(list(positional_entropies.items()), columns=['Player', 'Entropy'])

# Step 6: Passing Network
def build_passing_network(df, player_col='player_name', event_col='event_type', outcome_col='event_outcome'):
    pass_events = df[df[event_col] == 1]  # Example event type for passes
    G = nx.DiGraph()
    for i in range(len(pass_events) - 1):
        passer = pass_events.iloc[i][player_col]
        receiver = pass_events.iloc[i + 1][player_col]
        if passer != receiver:
            G.add_edge(passer, receiver, weight=1)
    centrality = nx.betweenness_centrality(G, weight='weight')
    return G, centrality

G, centrality = build_passing_network(df_cleaned)

# Save centrality metrics to a DataFrame
centrality_df = pd.DataFrame(list(centrality.items()), columns=['Player', 'Centrality'])

# Step 7: Predict Next Action
def train_action_predictor(df):
    df['time_in_seconds'] = df['timeMin'] * 60 + df['timeSec']
    features = df[['x_pos', 'y_pos', 'time_in_seconds']]
    labels = df['event_type']
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    print(f"Model Accuracy: {accuracy * 100:.2f}%")
    return model

model = train_action_predictor(df_cleaned)

# Step 8: Evaluate Team Style
def evaluate_team_style(entropies, centrality):
    avg_entropy = np.mean(list(entropies.values()))
    avg_centrality = np.mean(list(centrality.values()))
    return "Relationist" if avg_entropy > 1.5 and avg_centrality < 0.5 else "Positional"

team_style = evaluate_team_style(positional_entropies, centrality)
print(f"Team Style: {team_style}")

# Save team style to a DataFrame
team_style_df = pd.DataFrame([{'Metric': 'Team Style', 'Value': team_style}])

# Step 9: Save Results to Google Drive
output_file = os.path.join(drive_path, 'team_analysis_results.xlsx')

with pd.ExcelWriter(output_file) as writer:
    entropy_df.to_excel(writer, sheet_name='Positional Entropies', index=False)
    centrality_df.to_excel(writer, sheet_name='Centrality Metrics', index=False)
    team_style_df.to_excel(writer, sheet_name='Team Style', index=False)

print(f"All results have been saved to: {output_file}")


Mounted at /content/drive


Saving BM2425.xlsx to BM2425 (2).xlsx


<ipython-input-3-245959ceb670>:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  player_positions = df.groupby(player_col).apply(


Model Accuracy: 67.62%
Team Style: Relationist
All results have been saved to: /content/drive/My Drive/team_analysis_results.xlsx
